In [1]:
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd
import time

import tensorflow as tf
from tensorflow.keras.mixed_precision import Policy, set_global_policy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input, Dropout
from IPython.display import display, clear_output
from sklearn.preprocessing import MinMaxScaler
from IPython.display import display, clear_output
import time

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [4]:
%%time

# Use JIT Compilation
tf.config.optimizer.set_jit(True)

# Use mixed precision
# policy = Policy('mixed_float16')
# set_global_policy(policy)

with tf.device('/device:GPU:0'):

    # Function to create sequences
    # --------------------------------------

    def create_sequence(data, target, seq_length):
        Xs, ys = [], []
        for i in range(len(data) - seq_length):
            Xs.append(data[i:i + seq_length])
            ys.append(target[i + seq_length])
        return np.array(Xs), np.array(ys)

    ds = xr.open_dataset('/Users/smata/Downloads/processedData/velocityData.nc')

    # Reformat to Pandas dataframe
    # --------------------------------------

    df_u = ds['u'].to_dataframe().reset_index()
    df_v = ds['v'].to_dataframe().reset_index()

    df_u = df_u.pivot(index = 'time', columns = 'height', values = 'u')
    df_v = df_v.pivot(index = 'time', columns = 'height', values = 'v')

    df_u.columns = [f'u_{int(height)}m' for height in df_u.columns]
    df_v.columns = [f'v_{int(height)}m' for height in df_v.columns]

    df = pd.concat([df_u, df_v], axis = 1)
    df.reset_index(drop = True, inplace = True)

    df.insert(0, 'L',    ds.L.values)
    df.insert(0, 'TKE',  ds.TKE.values)

    df.insert(0, 'hour_sin', np.sin(2 * np.pi * ds.hr_day.values / 24))
    df.insert(0, 'hour_cos', np.cos(2 * np.pi * ds.hr_day.values / 24))

    df.insert(0, 'day_sin', np.sin(2 * np.pi * ds.day_yr.values / 365))
    df.insert(0, 'day_cos', np.cos(2 * np.pi * ds.day_yr.values / 365))

    # Standardize data
    # --------------------------------------

    windCols      = [col for col in df.columns if col.startswith('u_') or col.startswith('v_')]
    stabilityCols = ['TKE', 'L']

    wind_scaler = MinMaxScaler()
    df[windCols] = wind_scaler.fit_transform(df[windCols])

    stability_scaler = MinMaxScaler()
    df[stabilityCols] = stability_scaler.fit_transform(df[stabilityCols])

    # Define and extract input and output columns
    # --------------------------------------

    inputs  = ['day_cos', 'day_sin', 'hour_cos', 'hour_sin', 'TKE', 'L', 'u_10m', 'v_10m']

    X = df[inputs].values
    y = df[windCols].values

    # Create input and output arrays
    # --------------------------------------

    seq_length = 1
    X_sequence, y_sequence = create_sequence(X, y, seq_length)

    split_index = int(0.8 * len(X_sequence))

    X_train_sequence, X_test_sequence = X_sequence[:split_index], X_sequence[split_index:]
    y_train_sequence, y_test_sequence = y_sequence[:split_index], y_sequence[split_index:]

    batch = 64
            
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train_sequence, y_train_sequence))
    test_dataset = tf.data.Dataset.from_tensor_slices((X_test_sequence, y_test_sequence))

    train_dataset = train_dataset.cache().shuffle(buffer_size=1024).batch(batch).prefetch(tf.data.experimental.AUTOTUNE)
    test_dataset = test_dataset.cache().batch(batch).prefetch(tf.data.experimental.AUTOTUNE)

    input_shape = (seq_length, len(inputs))

    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(LSTM(64, return_sequences=True))
    model.add(LSTM(32))
    model.add(Dense(len(windCols)))

    model.compile(optimizer='adam', loss='mean_squared_error')

    history = model.fit(train_dataset, epochs=500, validation_data=test_dataset)

Epoch 1/500


2024-08-02 22:41:02.712974: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-08-02 22:41:02.914565: W tensorflow/compiler/jit/kernels/xla_ops.cc:466] Compilation failed:UNIMPLEMENTED: Could not find compiler for platform METAL: NOT_FOUND: could not find registered compiler for platform METAL -- check target linkage.  Falling back to TF function call.
2024-08-02 22:41:02.914578: W tensorflow/compiler/jit/kernels/xla_ops.cc:466] Compilation failed:UNIMPLEMENTED: Could not find compiler for platform METAL: NOT_FOUND: could not find registered compiler for platform METAL -- check target linkage.  Falling back to TF function call.
2024-08-02 22:41:02.914930: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-08-02 22:41:02.914943: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for d

 6/90 [=>............................] - ETA: 0s - loss: 0.3218  

2024-08-02 22:41:03.178325: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


90/90 [==============================] - ETA: 0s - loss: 0.1765

2024-08-02 22:41:04.347284: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-08-02 22:41:04.421049: W tensorflow/compiler/jit/kernels/xla_ops.cc:466] Compilation failed:UNIMPLEMENTED: Could not find compiler for platform METAL: NOT_FOUND: could not find registered compiler for platform METAL -- check target linkage.  Falling back to TF function call.
2024-08-02 22:41:04.421345: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-08-02 22:41:04.426647: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-08-02 22:41:04.474355: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


90/90 [==============================] - 3s 15ms/step - loss: 0.1765 - val_loss: 0.0284
Epoch 2/500
90/90 [==============================] - 1s 9ms/step - loss: 0.0158 - val_loss: 0.0205
Epoch 3/500
90/90 [==============================] - 1s 9ms/step - loss: 0.0142 - val_loss: 0.0200
Epoch 4/500
90/90 [==============================] - 1s 9ms/step - loss: 0.0138 - val_loss: 0.0195
Epoch 5/500
90/90 [==============================] - 1s 9ms/step - loss: 0.0134 - val_loss: 0.0184
Epoch 6/500
90/90 [==============================] - 1s 9ms/step - loss: 0.0128 - val_loss: 0.0178
Epoch 7/500
90/90 [==============================] - 1s 9ms/step - loss: 0.0125 - val_loss: 0.0177
Epoch 8/500
90/90 [==============================] - 1s 9ms/step - loss: 0.0121 - val_loss: 0.0167
Epoch 9/500
90/90 [==============================] - 1s 9ms/step - loss: 0.0116 - val_loss: 0.0158
Epoch 10/500
90/90 [==============================] - 1s 9ms/step - loss: 0.0111 - val_loss: 0.0154
Epoch 11/500
90/90 [

In [7]:
%%time

# Use JIT Compilation
# tf.config.optimizer.set_jit(True)

# Use mixed precision
# policy = Policy('mixed_float16')
# set_global_policy(policy)

with tf.device('/device:CPU:0'):

    tf.function(jit_compile=True)
    tf.config.run_functions_eagerly(False)

    # Function to create sequences
    # --------------------------------------

    def create_sequence(data, target, seq_length):
        Xs, ys = [], []
        for i in range(len(data) - seq_length):
            Xs.append(data[i:i + seq_length])
            ys.append(target[i + seq_length])
        return np.array(Xs), np.array(ys)

    ds = xr.open_dataset('/Users/smata/Downloads/processedData/velocityData.nc')

    # Reformat to Pandas dataframe
    # --------------------------------------

    df_u = ds['u'].to_dataframe().reset_index()
    df_v = ds['v'].to_dataframe().reset_index()

    df_u = df_u.pivot(index = 'time', columns = 'height', values = 'u')
    df_v = df_v.pivot(index = 'time', columns = 'height', values = 'v')

    df_u.columns = [f'u_{int(height)}m' for height in df_u.columns]
    df_v.columns = [f'v_{int(height)}m' for height in df_v.columns]

    df = pd.concat([df_u, df_v], axis = 1)
    df.reset_index(drop = True, inplace = True)

    df.insert(0, 'L',    ds.L.values)
    df.insert(0, 'TKE',  ds.TKE.values)

    df.insert(0, 'hour_sin', np.sin(2 * np.pi * ds.hr_day.values / 24))
    df.insert(0, 'hour_cos', np.cos(2 * np.pi * ds.hr_day.values / 24))

    df.insert(0, 'day_sin', np.sin(2 * np.pi * ds.day_yr.values / 365))
    df.insert(0, 'day_cos', np.cos(2 * np.pi * ds.day_yr.values / 365))

    # Standardize data
    # --------------------------------------

    windCols      = [col for col in df.columns if col.startswith('u_') or col.startswith('v_')]
    stabilityCols = ['TKE', 'L']

    wind_scaler = MinMaxScaler()
    df[windCols] = wind_scaler.fit_transform(df[windCols])

    stability_scaler = MinMaxScaler()
    df[stabilityCols] = stability_scaler.fit_transform(df[stabilityCols])

    # Define and extract input and output columns
    # --------------------------------------

    inputs  = ['day_cos', 'day_sin', 'hour_cos', 'hour_sin', 'TKE', 'L', 'u_10m', 'v_10m']

    X = df[inputs].values
    y = df[windCols].values

    # Create input and output arrays
    # --------------------------------------

    seq_length = 1
    X_sequence, y_sequence = create_sequence(X, y, seq_length)

    split_index = int(0.8 * len(X_sequence))

    X_train_sequence, X_test_sequence = X_sequence[:split_index], X_sequence[split_index:]
    y_train_sequence, y_test_sequence = y_sequence[:split_index], y_sequence[split_index:]

    batch = 64
            
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train_sequence, y_train_sequence))
    test_dataset = tf.data.Dataset.from_tensor_slices((X_test_sequence, y_test_sequence))

    train_dataset = train_dataset.cache().shuffle(buffer_size=1024).batch(batch).prefetch(tf.data.experimental.AUTOTUNE)
    test_dataset = test_dataset.cache().batch(batch).prefetch(tf.data.experimental.AUTOTUNE)

    input_shape = (seq_length, len(inputs))

    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(LSTM(64, return_sequences=True))
    model.add(LSTM(32))
    model.add(Dense(len(windCols)))

    model.compile(optimizer='adam', loss='mean_squared_error')

    history = model.fit(train_dataset, epochs=500, validation_data=test_dataset)

Epoch 1/500
90/90 [==============================] - 2s 5ms/step - loss: 0.1764 - val_loss: 0.0325
Epoch 2/500
90/90 [==============================] - 0s 1ms/step - loss: 0.0162 - val_loss: 0.0201
Epoch 3/500
90/90 [==============================] - 0s 1ms/step - loss: 0.0141 - val_loss: 0.0200
Epoch 4/500
90/90 [==============================] - 0s 1ms/step - loss: 0.0137 - val_loss: 0.0191
Epoch 5/500
90/90 [==============================] - 0s 1ms/step - loss: 0.0133 - val_loss: 0.0185
Epoch 6/500
90/90 [==============================] - 0s 1ms/step - loss: 0.0128 - val_loss: 0.0177
Epoch 7/500
90/90 [==============================] - 0s 1ms/step - loss: 0.0125 - val_loss: 0.0169
Epoch 8/500
90/90 [==============================] - 0s 1ms/step - loss: 0.0120 - val_loss: 0.0166
Epoch 9/500
90/90 [==============================] - 0s 1ms/step - loss: 0.0117 - val_loss: 0.0159
Epoch 10/500
90/90 [==============================] - 0s 1ms/step - loss: 0.0112 - val_loss: 0.0154
Epoch 11/

In [3]:
%%time

# Use JIT Compilation
tf.config.optimizer.set_jit(True)

# Use mixed precision
# policy = Policy('mixed_float16')
# set_global_policy(policy)

with tf.device('/device:GPU:0'):

    # Function to create sequences
    # --------------------------------------

    def create_sequence(data, target, seq_length):
        Xs, ys = [], []
        for i in range(len(data) - seq_length):
            Xs.append(data[i:i + seq_length])
            ys.append(target[i + seq_length])
        return np.array(Xs), np.array(ys)

    ds = xr.open_dataset('/Users/smata/Downloads/processedData/velocityData.nc')

    # Reformat to Pandas dataframe
    # --------------------------------------

    df_u = ds['u'].to_dataframe().reset_index()
    df_v = ds['v'].to_dataframe().reset_index()

    df_u = df_u.pivot(index = 'time', columns = 'height', values = 'u')
    df_v = df_v.pivot(index = 'time', columns = 'height', values = 'v')

    df_u.columns = [f'u_{int(height)}m' for height in df_u.columns]
    df_v.columns = [f'v_{int(height)}m' for height in df_v.columns]

    df = pd.concat([df_u, df_v], axis = 1)
    df.reset_index(drop = True, inplace = True)

    df.insert(0, 'L',    ds.L.values)
    df.insert(0, 'TKE',  ds.TKE.values)

    df.insert(0, 'hour_sin', np.sin(2 * np.pi * ds.hr_day.values / 24))
    df.insert(0, 'hour_cos', np.cos(2 * np.pi * ds.hr_day.values / 24))

    df.insert(0, 'day_sin', np.sin(2 * np.pi * ds.day_yr.values / 365))
    df.insert(0, 'day_cos', np.cos(2 * np.pi * ds.day_yr.values / 365))

    # Standardize data
    # --------------------------------------

    windCols      = [col for col in df.columns if col.startswith('u_') or col.startswith('v_')]
    stabilityCols = ['TKE', 'L']

    wind_scaler = MinMaxScaler()
    df[windCols] = wind_scaler.fit_transform(df[windCols])

    stability_scaler = MinMaxScaler()
    df[stabilityCols] = stability_scaler.fit_transform(df[stabilityCols])

    # Define and extract input and output columns
    # --------------------------------------

    inputs  = ['day_cos', 'day_sin', 'hour_cos', 'hour_sin', 'TKE', 'L', 'u_10m', 'v_10m']

    X = df[inputs].values
    y = df[windCols].values

    # Create input and output arrays
    # --------------------------------------

    seq_length = 1
    X_sequence, y_sequence = create_sequence(X, y, seq_length)

    split_index = int(0.8 * len(X_sequence))

    X_train_sequence, X_test_sequence = X_sequence[:split_index], X_sequence[split_index:]
    y_train_sequence, y_test_sequence = y_sequence[:split_index], y_sequence[split_index:]

    batch = 64
            
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train_sequence, y_train_sequence))
    test_dataset = tf.data.Dataset.from_tensor_slices((X_test_sequence, y_test_sequence))

    train_dataset = train_dataset.cache().shuffle(buffer_size=1024).batch(batch).prefetch(tf.data.experimental.AUTOTUNE)
    test_dataset = test_dataset.cache().batch(batch).prefetch(tf.data.experimental.AUTOTUNE)

    input_shape = (seq_length, len(inputs))

    model = Sequential()

    model.add(Input(shape=input_shape))
    model.add(LSTM(64, activation = 'tanh', recurrent_activation = 'sigmoid', recurrent_dropout = 0, unroll = False, use_bias = True, return_sequences=True))
    # model.add(LSTM(32))
    model.add(Dense(len(windCols)))

    model.compile(optimizer='adam', loss='mean_squared_error')

    history = model.fit(train_dataset, epochs=500, validation_data=test_dataset)

Metal device set to: Apple M3 Pro

systemMemory: 36.00 GB
maxCacheSize: 13.50 GB



2024-08-02 23:20:06.737356: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-08-02 23:20:06.737511: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/500


2024-08-02 23:20:07.858311: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-08-02 23:20:07.860246: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-08-02 23:20:07.984428: W tensorflow/compiler/jit/kernels/xla_ops.cc:466] Compilation failed:UNIMPLEMENTED: Could not find compiler for platform METAL: NOT_FOUND: could not find registered compiler for platform METAL -- check target linkage.  Falling back to TF function call.
2024-08-02 23:20:07.984553: W tensorflow/compiler/jit/kernels/xla_ops.cc:466] Compilation failed:UNIMPLEMENTED: Could not find compiler for platform METAL: NOT_FOUND: could not find registered compiler for platform METAL -- check target linkage.  Falling back to TF function call.
2024-08-02 23:20:07.984825: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-08

 1/90 [..............................] - ETA: 2:45 - loss: 0.3237

2024-08-02 23:20:08.888458: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


90/90 [==============================] - ETA: 0s - loss: 0.1692

2024-08-02 23:20:09.964740: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-08-02 23:20:10.008800: W tensorflow/compiler/jit/kernels/xla_ops.cc:466] Compilation failed:UNIMPLEMENTED: Could not find compiler for platform METAL: NOT_FOUND: could not find registered compiler for platform METAL -- check target linkage.  Falling back to TF function call.
2024-08-02 23:20:10.009112: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-08-02 23:20:10.013859: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


90/90 [==============================] - 3s 13ms/step - loss: 0.1692 - val_loss: 0.0508
Epoch 2/500
90/90 [==============================] - 1s 8ms/step - loss: 0.0256 - val_loss: 0.0297
Epoch 3/500
90/90 [==============================] - 1s 8ms/step - loss: 0.0211 - val_loss: 0.0265
Epoch 4/500
90/90 [==============================] - 1s 8ms/step - loss: 0.0205 - val_loss: 0.0261
Epoch 5/500
90/90 [==============================] - 1s 8ms/step - loss: 0.0202 - val_loss: 0.0257
Epoch 6/500
90/90 [==============================] - 1s 8ms/step - loss: 0.0200 - val_loss: 0.0256
Epoch 7/500
90/90 [==============================] - 1s 8ms/step - loss: 0.0197 - val_loss: 0.0257
Epoch 8/500
90/90 [==============================] - 1s 8ms/step - loss: 0.0196 - val_loss: 0.0257
Epoch 9/500
90/90 [==============================] - 1s 8ms/step - loss: 0.0195 - val_loss: 0.0257
Epoch 10/500
90/90 [==============================] - 1s 8ms/step - loss: 0.0194 - val_loss: 0.0257
Epoch 11/500
90/90 [

In [ ]:
# Predict
# --------------------------------------

predictions = model.predict(X_sequence)

predictions = wind_scaler.inverse_transform(predictions)

In [ ]:
# Plot sample profiles
# --------------------------------------

i = 2100

fig,ax = plt.subplots(1, 3, figsize = (15, 5), sharey = True)

divider = int(len(outputs)/2)

ax[0].plot(ds.u[:,i], ds.height, label = 'Data')
ax[0].plot(predictions[i,:divider], ds.height, label = 'Model')
ax[0].set_xlabel('$u$ component [m/s]')
ax[0].set_ylabel('$z$ [m]')

ax[1].plot(ds.v[:,i], ds.height, label = 'Data')
ax[1].plot(predictions[i,divider:], ds.height, label = 'Model')
ax[1].set_xlabel('$v$ component [m/s]')

ax[2].plot(((ds.u[:,i]**2 + ds.v[:,i]**2)**(1/2)), ds.height, label = 'Data')
ax[2].plot((predictions[i,divider:]**2 + predictions[i,:divider]**2)**(1/2), ds.height, label = 'Model')
ax[2].set_xlabel('Wind speed [m/s]')


In [ ]:
# Plot sample profiles
# --------------------------------------

i = 150

fig,ax = plt.subplots(1, 3, figsize = (15, 5), sharey = True)

divider = int(len(outputs)/2)

ax[0].plot(ds.u[:,i], ds.height, label = 'Data')
ax[0].plot(predictions[i,:divider], ds.height, label = 'Model')
ax[0].set_xlabel('$u$ component [m/s]')
ax[0].set_ylabel('$z$ [m]')

ax[1].plot(ds.v[:,i], ds.height, label = 'Data')
ax[1].plot(predictions[i,divider:], ds.height, label = 'Model')
ax[1].set_xlabel('$v$ component [m/s]')

ax[2].plot(((ds.u[:,i]**2 + ds.v[:,i]**2)**(1/2)), ds.height, label = 'Data')
ax[2].plot((predictions[i,divider:]**2 + predictions[i,:divider]**2)**(1/2), ds.height, label = 'Model')
ax[2].set_xlabel('Wind speed [m/s]')